# K-means clustering on USPS digits dataset

In [1]:
from sklearn.datasets import load_digits
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.manifold import TSNE

Load the dataset

In [ ]:
digits = load_digits()
digits.data.shape